In [30]:
import rclpy
from rclpy.node import Node
from std_msgs.msg import String
import threading
from rclpy.executors import MultiThreadedExecutor

In [31]:
class Publisher(Node):
    def __init__(self):
      # Constructor de inicio
      super().__init__('simple_publisher')
      topic_name = "/topic"
      self.publisher = self.create_publisher(String, topic_name, 10)
      self.timer = self.create_timer(1.0, self.timer_callback)
      self.get_logger().info('Publicador iniciado en {}'.format(topic_name))
      self.count = 0

    def timer_callback(self):
      # Función de timer para publicar periódicamente cada segundo
      msg = String()
      msg.data = "Mensaje No. {}".format(self.count)
      self.publisher.publish(msg)
      self.get_logger().info('Publicando: {}'.format(msg.data))
      self.count += 1

class Subscriber(Node):
    def __init__(self):
      # Constructor
      super().__init__('simple_subscriber')
      topic_name = "/topic"
      self.subscription = self.create_subscription(String, topic_name, self.listener_callback, 10)
      self.get_logger().info('Suscriptor escuchando topico: {}'.format(topic_name))

    def listener_callback(self, msg):
      # Función que se llama al recibir un mensaje
      self.get_logger().info(f'Recibido: "{msg.data}"')


In [32]:
if not rclpy.ok():
  rclpy.init(args=None)

In [33]:
executor = MultiThreadedExecutor()
thread = threading.Thread(target=executor.spin, daemon=True)
thread.start()

In [ ]:
pub_node = Publisher()
sub_node = Subscriber()
executor.add_node(sub_node)
executor.add_node(pub_node)

[INFO] [1762645927.100145464] [simple_publisher]: Publicador iniciado en /topic
[INFO] [1762645927.112333728] [simple_subscriber]: Suscriptor escuchando topico: /topic


True

[INFO] [1762645928.097006625] [simple_publisher]: Publicando: Mensaje No. 0
[INFO] [1762645928.101787082] [simple_subscriber]: Recibido: "Mensaje No. 0"
[INFO] [1762645929.097230154] [simple_publisher]: Publicando: Mensaje No. 1
[INFO] [1762645929.102098812] [simple_subscriber]: Recibido: "Mensaje No. 1"
[INFO] [1762645930.097414569] [simple_subscriber]: Recibido: "Mensaje No. 2"
[INFO] [1762645930.102018630] [simple_publisher]: Publicando: Mensaje No. 2
[INFO] [1762645931.097083034] [simple_subscriber]: Recibido: "Mensaje No. 3"
[INFO] [1762645931.102605283] [simple_publisher]: Publicando: Mensaje No. 3
[INFO] [1762645932.097496289] [simple_subscriber]: Recibido: "Mensaje No. 4"
[INFO] [1762645932.101073907] [simple_publisher]: Publicando: Mensaje No. 4
[INFO] [1762645933.099536284] [simple_publisher]: Publicando: Mensaje No. 5
[INFO] [1762645933.104242673] [simple_subscriber]: Recibido: "Mensaje No. 5"
[INFO] [1762645934.096820799] [simple_publisher]: Publicando: Mensaje No. 6
[INFO]

In [35]:
pub_node.destroy_node()
sub_node.destroy_node()

if rclpy.ok():
    rclpy.shutdown()

In [16]:
import rclpy
from rclpy.node import Node
from example_interfaces.srv import AddTwoInts
from rclpy.task import Future

In [23]:
class Server(Node):
    def __init__(self):
      # Constructor
      super().__init__('Server')
      server_name = '/sum_two_ints'
      self.srv = self.create_service(AddTwoInts, server_name, self.listener_callback)
      self.get_logger().info('Servicio disponible en {}'.format(server_name))

    def listener_callback(self, request:AddTwoInts.Request, response:AddTwoInts.Response):
      # Función que se llama al recibir un mensaje
      response.sum = request.a + request.b
      self.get_logger().info("Recibido: a={}, b={}, respuesta={}".format(request.a, request.b, response.sum))
      return response
    
class Client(Node):
    def __init__(self):
      # Constructor de inicio
      super().__init__('Client')

      service_name = "/sum_two_ints"
      self.a = 10
      self.b = 10

      self.client = self.create_client(AddTwoInts, service_name)

      while not self.client.wait_for_service(timeout_sec=1.0):
         self.get_logger().info('Esperando al servicio {}...'.format(service_name))
      self.timer = self.create_timer(1.0, self.timer_callback)
      self.get_logger().info('Cliente iniciado a {}'.format(service_name))
      self.count = 0

    def timer_callback(self):
      # Función de timer para enviar una solicitud periódicamente cada segundo
      self.count += 1
      request = AddTwoInts.Request()
      request.a = self.a
      request.b = self.b + self.count
      # --- Manda solicitud
      future = self.client.call_async(request)
      # --- Función que se ejecuta al recibir respuesta
      future.add_done_callback(self.callback_result)

    def callback_result(self, future:Future):
      # Función que se llama al recibir una respuesta a la solicitud enviada
      try:
         response = future.result()
         response:AddTwoInts.Response
         self.get_logger().info("Resultado recibido: {} + {} = {}".format(self.a, self.b + self.count, response.sum))
      except Exception as e:
         self.get_logger().error(f"Error al llamar al servicio: {0}")



In [24]:
if not rclpy.ok():
    rclpy.init(args=None)

In [25]:
executor = MultiThreadedExecutor()
thread = threading.Thread(target=executor.spin, daemon=True)
thread.start()

In [26]:
server_node = Server()
client_node = Client()

[INFO] [1762645477.643877644] [Server]: Servicio disponible en /sum_two_ints
[INFO] [1762645477.658351388] [Client]: Cliente iniciado a /sum_two_ints


In [27]:
executor.add_node(server_node)
executor.add_node(client_node)

True

[INFO] [1762645479.457548387] [Server]: Recibido: a=10, b=11, respuesta=21
[INFO] [1762645479.467241592] [Client]: Resultado recibido: 10 + 11 = 21
[INFO] [1762645479.668476095] [Server]: Recibido: a=10, b=12, respuesta=22
[INFO] [1762645479.692372210] [Client]: Resultado recibido: 10 + 12 = 22
[INFO] [1762645480.670024094] [Server]: Recibido: a=10, b=13, respuesta=23
[INFO] [1762645480.691380210] [Client]: Resultado recibido: 10 + 13 = 23
[INFO] [1762645481.666431264] [Server]: Recibido: a=10, b=14, respuesta=24
[INFO] [1762645481.686000592] [Client]: Resultado recibido: 10 + 14 = 24
[INFO] [1762645482.665122591] [Server]: Recibido: a=10, b=15, respuesta=25
[INFO] [1762645482.682080323] [Client]: Resultado recibido: 10 + 15 = 25
[INFO] [1762645483.665635224] [Server]: Recibido: a=10, b=16, respuesta=26
[INFO] [1762645483.683131282] [Client]: Resultado recibido: 10 + 16 = 26
[INFO] [1762645484.660022353] [Server]: Recibido: a=10, b=17, respuesta=27
[INFO] [1762645484.664831155] [Client

In [28]:
server_node.destroy_node()
client_node.destroy_node()

if rclpy.ok():
  rclpy.shutdown()